<a href="https://colab.research.google.com/github/Kinrider/TechChallenge/blob/main/Estima_Producao_espumantes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importando Bibliotecas

In [1]:
import pandas as pd
import random as rd

# Define as opções para exibir floats sem notação científica

pd.set_option('display.float_format', '{:.0f}'.format)

# Tratamento de dados de Exportação

In [6]:
# Para ler arquivos do Git, sempre específicar no final : '?raw=true'

dados = 'https://github.com/Kinrider/TechChallenge/blob/91c7c009e289c34462dfc3ced5d99ba76e818a36/Fontes/ExpEspumantes.csv?raw=true'

In [7]:
vinhosexp = pd.read_csv(dados, sep=';' , encoding= 'utf8')
vinhosexp
x = vinhosexp.copy()

In [8]:
x.head()

,Id,País,1970,1970.1,1971,1971.1,1972,1972.1,1973,1973.1,...,2019,2019.1,2020,2020.1,2021,2021.1,2022,2022.1,2023,2023.1
0,1,África do Sul,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,44
1,2,Alemanha,0,0,0,0,0,0,0,0,...,1003,5466,2388,14767,142,265,1164,6560,162,1542
2,3,Angola,0,0,0,0,0,0,0,0,...,1007,3615,24,38,0,0,26383,141588,56242,315073
3,4,Antigua e Barbuda,0,0,0,0,0,0,0,0,...,7,34,32,328,10,82,65,146,24,100
4,5,Antilhas Holandesas,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
array_dollar=['Id' , 'País']
array_kilos=['Id' , 'País']

for column in x.columns:
  if '.1' in column:
   dollar = column.replace('.1' , '')+'_'+'dollar'
   x.rename({column:dollar},inplace=True, axis=1)
   array_dollar.append(dollar)
  else:
    kilos = column +'_'+ 'kilos'
    x.rename({column:kilos},inplace=True, axis=1)
    array_kilos.append(kilos)

In [10]:
x.rename({'Id_kilos' :'Id'},inplace=True, axis=1)
x.rename({'País_kilos':'País'},inplace=True, axis=1)

In [11]:
array_kilos.remove('Id_kilos')
array_kilos.remove('País_kilos')

In [12]:
x[array_kilos].head()

,Id,País,1970_kilos,1971_kilos,1972_kilos,1973_kilos,1974_kilos,1975_kilos,1976_kilos,1977_kilos,...,2014_kilos,2015_kilos,2016_kilos,2017_kilos,2018_kilos,2019_kilos,2020_kilos,2021_kilos,2022_kilos,2023_kilos
0,1,África do Sul,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
1,2,Alemanha,0,0,0,0,0,0,0,0,...,9236,6256,4293,4824,4092,1003,2388,142,1164,162
2,3,Angola,0,0,0,0,0,0,0,0,...,6866,0,90,0,63,1007,24,0,26383,56242
3,4,Antigua e Barbuda,0,0,0,0,0,0,0,0,...,0,0,0,0,0,7,32,10,65,24
4,5,Antilhas Holandesas,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
x[array_dollar].head()

,Id,País,1970_dollar,1971_dollar,1972_dollar,1973_dollar,1974_dollar,1975_dollar,1976_dollar,1977_dollar,...,2014_dollar,2015_dollar,2016_dollar,2017_dollar,2018_dollar,2019_dollar,2020_dollar,2021_dollar,2022_dollar,2023_dollar
0,1,África do Sul,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,44
1,2,Alemanha,0,0,0,0,0,0,0,0,...,62169,30939,19088,28799,21373,5466,14767,265,6560,1542
2,3,Angola,0,0,0,0,0,0,0,0,...,22423,0,1010,0,280,3615,38,0,141588,315073
3,4,Antigua e Barbuda,0,0,0,0,0,0,0,0,...,0,0,0,0,0,34,328,82,146,100
4,5,Antilhas Holandesas,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
df = x

In [15]:
# Reshape do dataframe

# 1º transpor kilos para linha

df1 = df.melt(
    id_vars = ['Id', 'País'],
    value_vars = df.filter(like = '_kilos').columns,
    var_name = 'ano',
    value_name = 'kilos'
    )

# 2º transpor dólar para linha

df2 = df.melt(
    id_vars = ['Id', 'País'],
    value_vars = df.filter(like = '_dollar').columns,
    var_name = 'anos',
    value_name = 'dollar'
    )

# 3º sincronizar os índices

df1 = df1.set_index(['Id','País', df1.groupby(['Id','País']).cumcount()])
df2 = df2.set_index(['Id','País', df1.groupby(['Id','País']).cumcount()])

# 4º Juntar data_frames

df3 = (
    pd.concat([df1, df2], axis=1)
  .sort_index(level=2)
  .reset_index(level=2, drop=True)
  .reset_index()
  )

df3.head()

# referências:
# https://stackoverflow.com/questions/51519101/simultaneously-melt-multiple-columns-in-python-pandas
# https://pandas.pydata.org/docs/reference/api/pandas.melt.html
# https://medium.com/horadecodar/data-science-tips-02-como-usar-loc-e-iloc-no-pandas-fab58e214d87

,Id,País,ano,kilos,anos,dollar
0,1,África do Sul,1970_kilos,0,1970_dollar,0
1,2,Alemanha,1970_kilos,0,1970_dollar,0
2,3,Angola,1970_kilos,0,1970_dollar,0
3,4,Antigua e Barbuda,1970_kilos,0,1970_dollar,0
4,5,Antilhas Holandesas,1970_kilos,0,1970_dollar,0


In [16]:
# Dropa coluna de ano duplicada

df3.drop(['anos'], axis=1, inplace = True)

In [17]:
# tratando o campo de ano

df3['ano'] = df3['ano'].str.replace('_kilos','') + '-01-01'

df3.head()

,Id,País,ano,kilos,dollar
0,1,África do Sul,1970-01-01,0,0
1,2,Alemanha,1970-01-01,0,0
2,3,Angola,1970-01-01,0,0
3,4,Antigua e Barbuda,1970-01-01,0,0
4,5,Antilhas Holandesas,1970-01-01,0,0


In [18]:
# tranformando string em data

df3['ano'] = pd.to_datetime(df3['ano'],format='%Y-%m-%d')

df3.head()

,Id,País,ano,kilos,dollar
0,1,África do Sul,1970-01-01,0,0
1,2,Alemanha,1970-01-01,0,0
2,3,Angola,1970-01-01,0,0
3,4,Antigua e Barbuda,1970-01-01,0,0
4,5,Antilhas Holandesas,1970-01-01,0,0


In [19]:
# fitrando dados de 2008 pra frente:

df3.query('ano >= "2008-01-01"' ,inplace = True)

In [20]:
df3.head()

,Id,País,ano,kilos,dollar
3914,1,África do Sul,2008-01-01,0,0
3915,2,Alemanha,2008-01-01,5423,20928
3916,3,Angola,2008-01-01,2244,10581
3917,4,Antigua e Barbuda,2008-01-01,0,0
3918,5,Antilhas Holandesas,2008-01-01,0,0


In [21]:
# Verificando nomenclatura dos países

df3['País'].unique()

array(['África do Sul', 'Alemanha', 'Angola', 'Antigua e Barbuda',
       'Antilhas Holandesas', 'Argentina', 'Aruba', 'Australia',
       'Bahamas', 'Bangladesh', 'Barbados', 'Belgica', 'Benin',
       'Bermudas', 'Bolívia', 'Bósnia-Herzegovina', 'Bulgaria',
       'Cabo Verde', 'Camarões', 'Canada', 'Catar', 'Cayman, Ilhas',
       'Chile', 'China', 'Chipre', 'Cingapura', 'Colombia',
       'Coreia do Sul, Republica da', 'Costa Rica', 'Cuba', 'Curaçao',
       'Dinamarca', 'Dominica', 'El Salvador', 'Emirados Arabes Unidos',
       'Equador', 'Espanha', 'Estados Unidos', 'Estonia',
       'Falkland (Malvinas)', 'Filipinas', 'Filânldia', 'França', 'Gana',
       'Gibraltar', 'Granada', 'Grécia', 'Guatemala', 'Guiana',
       'Guiné Equatorial', 'Guiné-Bissau', 'Haiti', 'Honduras',
       'Hong Kong', 'Hungria', 'Ilha de Man', 'Índia', 'Iraque',
       'Irlanda', 'Islândia', 'Itália', 'Japão', 'Jordânia', 'Letônia',
       'Líbano', 'Libéria', 'Luxemburgo', 'Maldivas', 'Malta',
       

In [22]:
# Solicitado ao GPT que gerasse a lista de de_para, considerando os países listados acima

lista_paises = [
    ('Afeganistão', 'Afghanistan', 'AFG'),
    ('África do Sul', 'South Africa', 'ZAF'),
    ('Alemanha, República Democrática', 'Germany', 'DEU'),
    ('Angola', 'Angola', 'AGO'),
    ('Anguilla', 'Anguilla', 'AIA'),
    ('Antígua e Barbuda', 'Antigua and Barbuda', 'ATG'),
    ('Antilhas Holandesas', 'Netherlands Antilles', 'ANT'),
    ('Arábia Saudita', 'Saudi Arabia', 'SAU'),
    ('Argentina', 'Argentina', 'ARG'),
    ('Aruba', 'Aruba', 'ABW'),
    ('Austrália', 'Australia', 'AUS'),
    ('Áustria', 'Austria', 'AUT'),
    ('Bahamas', 'Bahamas', 'BHS'),
    ('Bangladesh', 'Bangladesh', 'BGD'),
    ('Barbados', 'Barbados', 'BRB'),
    ('Barein', 'Bahrain', 'BHR'),
    ('Bélgica', 'Belgium', 'BEL'),
    ('Belice', 'Belize', 'BLZ'),
    ('Benin', 'Benin', 'BEN'),
    ('Bermudas', 'Bermuda', 'BMU'),
    ('Bolívia', 'Bolivia', 'BOL'),
    ('Bósnia-Herzegovina', 'Bosnia and Herzegovina', 'BIH'),
    ('Brasil', 'Brazil', 'BRA'),
    ('Bulgária', 'Bulgaria', 'BGR'),
    ('Cabo Verde', 'Cabo Verde', 'CPV'),
    ('Camarões', 'Cameroon', 'CMR'),
    ('Canadá', 'Canada', 'CAN'),
    ('Catar', 'Qatar', 'QAT'),
    ('Cayman, Ilhas', 'Cayman Islands', 'CYM'),
    ('Chile', 'Chile', 'CHL'),
    ('China', 'China, mainland', 'CHN'),
    ('Chipre', 'Cyprus', 'CYP'),
    ('Cingapura', 'Singapore', 'SGP'),
    ('Cocos (Keeling), Ilhas', 'Cocos (Keeling) Islands', 'CCK'),
    ('Colômbia', 'Colombia', 'COL'),
    ('Comores', 'Comoros', 'COM'),
    ('Congo', 'Congo', 'COG'),
    ('Coreia, Republica Sul', 'Korea (the Republic of)', 'KOR'),
    ('Costa do Marfim', "Côte d'Ivoire", 'CIV'),
    ('Costa Rica', 'Costa Rica', 'CRI'),
    ('Croácia', 'Croatia', 'HRV'),
    ('Cuba', 'Cuba', 'CUB'),
    ('Curaçao', 'Curacao', 'CUW'),
    ('Dinamarca', 'Denmark', 'DNK'),
    ('Dominica', 'Dominica', 'DMA'),
    ('El Salvador', 'El Salvador', 'SLV'),
    ('Emirados Arabes Unidos', 'United Arab Emirates', 'ARE'),
    ('Equador', 'Ecuador', 'ECU'),
    ('Eslovaca, Republica', 'Slovakia', 'SVK'),
    ('Espanha', 'Spain', 'ESP'),
    ('Estados Unidos', 'United States of America', 'USA'),
    ('Estônia', 'Estonia', 'EST'),
    ('Filipinas', 'Philippines', 'PHL'),
    ('Finlândia', 'Finland', 'FIN'),
    ('França', 'France', 'FRA'),
    ('Gana', 'Ghana', 'GHA'),
    ('Gibraltar', 'Gibraltar', 'GIB'),
    ('Granada', 'Grenada', 'GRD'),
    ('Grécia', 'Greece', 'GRC'),
    ('Guatemala', 'Guatemala', 'GTM'),
    ('Guiana', 'Guyana', 'GUY'),
    ('Guiana Francesa', 'French Guiana', 'GUF'),
    ('Guine Bissau', 'Guinea-Bissau', 'GNB'),
    ('Guine Equatorial', 'Equatorial Guinea', 'GNQ'),
    ('Haiti', 'Haiti', 'HTI'),
    ('Honduras', 'Honduras', 'HND'),
    ('Hong Kong', 'China, Hong Kong SAR', 'HKG'),
    ('Hungria', 'Hungary', 'HUN'),
    ('Ilha de Man', 'Isle of Man', 'IMN'),
    ('Ilhas Virgens', 'Virgin Islands', 'VIR'),
    ('India', 'India', 'IND'),
    ('Indonésia', 'Indonesia', 'IDN'),
    ('Irã', 'Iran', 'IRN'),
    ('Iraque', 'Iraq', 'IRQ'),
    ('Irlanda', 'Ireland', 'IRL'),
    ('Itália', 'Italy', 'ITA'),
    ('Jamaica', 'Jamaica', 'JAM'),
    ('Japão', 'Japan', 'JPN'),
    ('Jordânia', 'Jordan', 'JOR'),
    ('Letônia', 'Latvia', 'LVA'),
    ('Líbano', 'Lebanon', 'LBN'),
    ('Libéria', 'Liberia', 'LBR'),
    ('Luxemburgo', 'Luxemburg', 'LUX'),
    ('Macau', 'China, Macao SAR', 'MAC'),
    ('Malásia', 'Malaysia', 'MYS'),
    ('Malavi', 'Malawi', 'MWI'),
    ('Malta', 'Malta', 'MLT'),
    ('Marshall, Ilhas', 'Marshall Islands', 'MHL'),
    ('Martinica', 'Martinique', 'MTQ'),
    ('Mauritânia', 'Mauritania', 'MRT'),
    ('México', 'Mexico', 'MEX'),
    ('Moçambique', 'Mozambique', 'MOZ'),
    ('Montenegro', 'Montenegro', 'MNE'),
    ('Namíbia', 'Namibia', 'NAM'),
    ('Nicarágua', 'Nicaragua', 'NIC'),
    ('Nigéria', 'Nigeria', 'NGA'),
    ('Noruega', 'Norway', 'NOR'),
    ('Nova Caledônia', 'New Caledonia', 'NCL'),
    ('Nova Zelândia', 'New Zealand', 'NZL'),
    ('Omã', 'Oman', 'OMN'),
    ('Países Baixos', 'Netherlands', 'NLD'),
    ('Palau', 'Palau', 'PLW'),
    ('Panamá', 'Panama', 'PAN'),
    ('Paraguai', 'Paraguay', 'PRY'),
    ('Peru', 'Peru', 'PER'),
    ('Pitcairn', 'Pitcairn Islands', 'PCN'),
    ('Polônia', 'Poland', 'POL'),
    ('Porto Rico', 'Puerto Rico', 'PRI'),
    ('Portugal', 'Portugal', 'PRT'),
    ('Quênia', 'Kenya', 'KEN'),
    ('Reino Unido', 'United Kingdom', 'GBR'),
    ('República Dominicana', 'Dominican Republic', 'DOM'),
    ('Rússia', 'Russia', 'RUS'),
    ('São Cristóvão e Névis', 'Saint Kitts and Nevis', 'KNA'),
    ('São Tomé e Príncipe', 'Sao Tome and Principe', 'STP'),
    ('São Vicente e Granadinas', 'Saint Vincent and the Grenadines', 'VCT'),
    ('Senegal', 'Senegal', 'SEN'),
    ('Serra Leoa', 'Sierra Leone', 'SLE'),
    ('Singapura', 'Singapore', 'SGP'),
    ('Suazilândia', 'Eswatini', 'SWZ'),
    ('Suécia', 'Sweden', 'SWE'),
    ('Suíça', 'Switzerland', 'CHE'),
    ('Suriname', 'Suriname', 'SUR'),
    ('Tailândia', 'Thailand', 'THA'),
    ('Taiwan (Formosa)', 'China, Taiwan Province of', 'TWN'),
    ('Tanzânia', 'Tanzania, the United Republic of', 'TZA'),
    ('Tcheca, República', 'Czech Republic', 'CZE'),
    ('Togo', 'Togo', 'TGO'),
    ('Toquelau', 'Tokelau', 'TKL'),
    ('Trinidade Tobago', 'Trinidad and Tobago', 'TTO'),
    ('Tunísia', 'Tunisia', 'TUN'),
    ('Turquia', 'Republic of Türkiye', 'TUR'),
    ('Tuvalu', 'Tuvalu', 'TUV'),
    ('Uruguai', 'Uruguay', 'URY'),
    ('Vanuatu', 'Vanuatu', 'VUT'),
    ('Venezuela', 'Venezuela', 'VEN'),
    ('Vietnã', 'Vietnam', 'VNM')
]

# Tranformando a tupla de países em um dataframe:

de_para_pais = pd.DataFrame(lista_paises, columns = ['pais_pt','pais_in','cod_pais'])

In [23]:
# join com de_para de paises

df_exp = df3.merge(de_para_pais, how = 'left', left_on=['País'],right_on = ['pais_pt'])

In [24]:
# Dropa colunas não utilizadas:

df_exp.drop(['País','pais_in','Id'], axis=1, inplace = True)

In [25]:
df_exp.head()

,ano,kilos,dollar,pais_pt,cod_pais
0,2008-01-01,0,0,África do Sul,ZAF
1,2008-01-01,5423,20928,NaN,NaN
2,2008-01-01,2244,10581,Angola,AGO
3,2008-01-01,0,0,NaN,NaN
4,2008-01-01,0,0,Antilhas Holandesas,ANT


#Estimando Produção Ventti della valli

In [26]:
df_exp.head()

,ano,kilos,dollar,pais_pt,cod_pais
0,2008-01-01,0,0,África do Sul,ZAF
1,2008-01-01,5423,20928,NaN,NaN
2,2008-01-01,2244,10581,Angola,AGO
3,2008-01-01,0,0,NaN,NaN
4,2008-01-01,0,0,Antilhas Holandesas,ANT


In [27]:
# Define uma função que vai aplicar sobre o total um percentual entre 18% a 24% do volume total para a nossa vinícola

def mult(x):
  return (rd.randint(2, 4)/100) * x

In [28]:
# Reinicia o índice para futura iteração

df_exp.reset_index(drop=True)

,ano,kilos,dollar,pais_pt,cod_pais
0,2008-01-01,0,0,África do Sul,ZAF
1,2008-01-01,5423,20928,NaN,NaN
2,2008-01-01,2244,10581,Angola,AGO
3,2008-01-01,0,0,NaN,NaN
4,2008-01-01,0,0,Antilhas Holandesas,ANT
...,...,...,...,...,...
1643,2023-01-01,2812,14352,Uruguai,URY
1644,2023-01-01,0,0,Vanuatu,VUT
1645,2023-01-01,0,0,Venezuela,VEN
1646,2023-01-01,0,0,Vietnã,VNM


In [29]:
#Itera sobre o index, de forma a aplicar para cada index na coluna criada chamada produção a função de multiplicação

for index, row in df_exp.iterrows():
  # Atualiza a célula da linha atual (index) e coluna 'producao' usando loc
  df_exp.loc[index, 'producao'] = mult(row['kilos'])

In [30]:
df_exp

,ano,kilos,dollar,pais_pt,cod_pais,producao
0,2008-01-01,0,0,África do Sul,ZAF,0
1,2008-01-01,5423,20928,NaN,NaN,163
2,2008-01-01,2244,10581,Angola,AGO,90
3,2008-01-01,0,0,NaN,NaN,0
4,2008-01-01,0,0,Antilhas Holandesas,ANT,0
...,...,...,...,...,...,...
1643,2023-01-01,2812,14352,Uruguai,URY,56
1644,2023-01-01,0,0,Vanuatu,VUT,0
1645,2023-01-01,0,0,Venezuela,VEN,0
1646,2023-01-01,0,0,Vietnã,VNM,0


In [ ]:
#dropa colunas não utilizada

df_exp.drop(['kilos','dollar'], axis = 1, inplace = True)

# Salva Arquivos Gerados

In [ ]:
##Salva Arquivo

#saída em csv
#df_exp.to_csv('/content/drive/Othercomputers/Meu laptop/Fiap - Scripts/01 - Tech_Challenge/01 - Dados/Dados_tratados.csv',sep = ';' , encoding='utf-8', index = False)

#saída em excel
df_exp.to_excel('/content/drive/Othercomputers/Meu laptop/Fiap - Scripts/01 - Tech_Challenge/01 - Dados/producao_vinho_tinto.xlsx', index = False)